`Meeting Notes`

- simplying data: confirm core fields
- heterogeneity as graph name?
- price of electricity is 0
- 'Canada.Alberta.Residential.Buildingss(Apartment)'
- make sure LCC components are correct

In [1]:
# !jupyter nbconvert --to script network_maude_sim.ipynb

In [2]:
%matplotlib inline

import pandas as pd
import re
from IPython.display import Image, display, Markdown, HTML
import copy
from pprint import pprint
import helpers
import networkx as nx

import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt

from helper.utils import * # for functions `process_subtypes`, `filter_fun`, `init_logger`

In [3]:
# Start a log file (from utils module)
init_logger(filename="supply_demand")

Directory does not exist, creating a new directory named /log/2019-11-29/...



In [4]:
# ************************
# User Defined Parameters
# ************************
infile = "../../pyCIMS_model_description.xlsm"
NODE_COL = "Node"
TYPE_COL = "Demand?"
MODEL_SHEET = "Model"
EXTRA_COL = 'Demand?'


# ************************
# Read in the data
# ************************

# Read model_description from excel
mxl = pd.read_excel(infile, sheet_name=None, header=1)

# Read the model sheet into a dataframe
model_df = mxl[MODEL_SHEET].replace({pd.np.nan: None})

# Adjust index to correspond to Excel line numbers
model_df.index += 3  #(+1: 0 vs 1 origin, +1: header skip, +1: column headers)

# Convert all column names to strings (years were ints)
model_df.columns = [str(c) for c in model_df.columns]

# Find the columns, separated by whether they are year columns or not
node_cols, year_cols = helpers.get_node_cols(model_df, extra_col=EXTRA_COL)
all_cols = np.concatenate((node_cols, year_cols))

# Create the model dataframe
mdf = model_df.loc[1:,all_cols] # drop irrelevant columns and skip first, empty row

In [5]:
# ************************
# Extract Node Dataframes
# ************************
# determine, row ranges for each node def, based on non-empty Node field
node_rows = mdf.Node[~mdf.Node.isnull()] # does not work if node names have been filled in
node_rows.index.name = "Row Number"
last_row = mdf.index[-1]
node_start_ends = zip(node_rows.index,
                      node_rows.index[1:].tolist() + [last_row])

# extract Node DataFrames, at this point still including Technologies
node_dfs = {}
non_node_cols = mdf.columns != NODE_COL
for s, e in node_start_ends:
#     node_name = mdf.Node[s]
    node_df = mdf.loc[s+1:e-1]
    node_df = node_df.loc[helpers.non_empty_rows(node_df), non_node_cols]
    
    try:
        node_name = list(node_df[node_df['Parameter']=='Service provided']['Branch'])[0]
    except IndexError:
        continue   

    node_dfs[node_name] = node_df

In [6]:
# ************************
# Extract Tech Dataframes
# ************************
# Extract tech dfs from node df's and rewrite node df without techs
tech_dfs = {}
for nn, ndf in node_dfs.items():
    if any(ndf.Parameter.isin(["Technology", "Service"])):  # Technologies can also be called Services
        tdfs = {}
        first_row, last_row = ndf.index[0], ndf.index[-1]
        tech_rows = ndf.loc[ndf.Parameter.isin(["Technology", "Service"])].index
        for trs, tre in zip(tech_rows, tech_rows[1:].tolist()+[last_row]):
            tech_df = ndf.loc[trs:tre-1]
            tech_name = tech_df.iloc[0].Value
            tdfs[tech_name] = tech_df
        tech_dfs[nn] = tdfs
        node_dfs[nn] = ndf.loc[:tech_rows[0]-1]
        
# see last node
node_df.head()        

,Parameter,Source,Branch,Unit,Value,Demand?,2000,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050
1130,Service provided,None,Canada.Alberta.Water,GJ,Water,Supply,None,None,None,None,None,None,None,None,None,None,None
1131,None,None,None,None,None,Supply,None,None,None,None,None,None,None,None,None,None,None
1132,None,None,None,None,None,Supply,None,None,None,None,None,None,None,None,None,None,None
1133,None,None,None,None,None,Supply,None,None,None,None,None,None,None,None,None,None,None
1134,None,None,None,None,None,Supply,None,None,None,None,None,None,None,None,None,None,None


In [7]:
# ************************
# Functions
# ************************
def add_node_data(g, current_node,):
    """
    @param g: NetworkX DiGraph. 
    @param current_node: The 
    @return: None. Modified the graph g instead. 
    Given a graph g and node current_node add current_node to g, along with all of its associated data."""

    # Copy the current node dataframe
    current_node_df = copy.deepcopy(node_dfs[current_node])

    # 1. we are going to create a node in the graph
    g.add_node(current_node)

    # 2. We will store the Supply/Demand Type of node. This is a special case. 
    typ = list(current_node_df[TYPE_COL])[0]
    g.node[current_node]['type'] = typ.lower() if typ else 'standard'
    # Drop Demand column
    current_node_df = current_node_df.drop(TYPE_COL, axis=1)

    # 3. We will store the Competition Type of the node at the node level. This is another special case. 
    comp_list = list(current_node_df[current_node_df['Parameter']=='Competition type']['Value'])
    if len(set(comp_list)) == 1: 
        comp_type = comp_list[0]
        g.node[current_node]['competition_type'] = comp_type.lower()
    elif len(set(comp_list)) > 1:
        print("TOO MANY COMPETITION TYPES")
    # Get rid of competition type row
    current_node_df = current_node_df[current_node_df['Parameter']!='Competition type']

    # 4. For the remaining rows, group data by year. 
    # Get Year Columns
    years = [c for c in current_node_df.columns if helpers.is_year(c)]

    # Get Non-Year Columns
    non_years = [c for c in current_node_df.columns if not helpers.is_year(c)]

    # For each year: 
    for y in years:
        year_df = current_node_df[non_years + [y]]
        year_dict = {}

        for parameter, source, branch, unit, value, year_value in zip(*[year_df[c] for c in year_df.columns]):
            if parameter in year_dict.keys():
                pass
            else:
                year_dict[parameter] = {}

            dct = {'source': source,
                   'branch': branch, 
                   'unit': unit, 
                   'year_value': year_value}
#             # Clean Dict
#             clean_dict = {k: v for k, v in dct.items() if v is not None}
            
            year_dict[parameter][value] = dct
            
        # Add data to node
        g.node[current_node][y] = year_dict
    
    
def add_tech_data(node, tech, tech_df):
    # TODO: I think we need to differentiate between Technologies and Services. 

    t_df = copy.deepcopy(tech_df)
    
    # 1. Find whether technology is a service or a technology. 
    #    Then remove the row that indicates this is a service or technology. 
    service_technology = 'service' if (t_df['Parameter']=='Service').any() else 'technology'
    t_df = t_df[~t_df['Parameter'].isin(['Service', 'Technology'])]

    # 2. Remove the Demand? column
    t_df = t_df.drop('Demand?', axis=1)

    # VERY SIMILAR to what we do for nodes. But not quite. Because we don't use the value column anymore
    # 4. For the remaining rows, group data by year. 
    # Get Year Columns
    years = [c for c in t_df.columns if helpers.is_year(c)]

    # Get Non-Year Columns
    non_years = [c for c in t_df.columns if not helpers.is_year(c)]

    # For each year: 
    for y in years:
        year_df = t_df[non_years + [y]]
        year_dict = {}

        for parameter, source, branch, unit, value, year_value in zip(*[year_df[c] for c in year_df.columns]):
            dct = {'source': source,
                       'branch': branch, 
                       'unit': unit, 
                       'year_value': year_value}
            
            if parameter in year_dict.keys():
                if type(year_dict[parameter]) is list:
                    year_dict[parameter] = year_dict[parameter].append(dct)
                else: 
                    year_dict[parameter] = [year_dict[parameter], dct] 
            else:
                year_dict[parameter] = dct

        # Add technologies key if needed
        if not 'technologies' in g.node[node][y].keys():
            g.node[node][y]['technologies'] = {}
            
        # Add the technology specific data for that year
        g.node[node][y]['technologies'][tech]= year_dict
        
def search_nodes(g, search_term):
    """Search nodes to see if there is one that contains the search term in the final component of its name"""
    def search(name):
        components = name.split('.')
        last_comp = components[-1]
        
        return search_term.lower() in last_comp.lower()
    
    return [n for n in g.nodes if search(n)]


In [8]:
# ************************
# Add Node Data
# ************************

# Create Graph
g = nx.DiGraph()

# Add each node and its associated data to the Graph
for n in node_dfs.keys():
    add_node_data(g, n)
#     print("************* {} *************".format(n))
#     pprint(g.node[n])

In [9]:
for node in tech_dfs:
    # Add technologies key to node data
    node_techs_dict = {}
    for tech in tech_dfs[node]: 
        add_tech_data(node, tech, tech_dfs[node][tech])

In [10]:
space_heating = search_nodes(g, 'space heating')[0]

In [11]:
def add_edges(node, df, g):
    # Find edges based on Requester/Provider relationships
    # ----------------------------------------------------
    # Find all nodes node is requesting services from
    providers = df[df['Parameter']=='Service requested']['Branch'].unique()
    rp_edges = [(node, p) for p in providers]
    g.add_edges_from(rp_edges)
    
    # Add them to the graph
    for e in rp_edges:
        try:
            types = g.edges[e]['type']
            if not 'request_provide' in types:
                g.edges[e]['type'] += ['request_provide']
        except KeyError:
            g.edges[e]['type'] = ['request_provide']
            

    # Find edges based on branch
    # --------------------------
    # Find the node's parent
    parent = '.'.join(node.split('.')[:-1])
    s_edges = []
    if parent: 
        s_edges += [(parent, node)]    
    g.add_edges_from(s_edges)
        
    # Add them to the graph
    for e in s_edges:
        try:
            types = g.edges[e]['type']
            if not 'structure' in types:
                g.edges[e]['type'] += ['structure']
        except KeyError:
            g.edges[e]['type'] = ['structure']  

In [12]:
for node in node_dfs:
    add_edges(node, node_dfs[node], g)
    
for node in tech_dfs:
    for tech in tech_dfs[node]:
        add_edges(node, tech_dfs[node][tech], g)

In [13]:
# Find the demand sub-graph

d_nodes = []
for n, a in g.nodes(data=True):
    try:
        a['type']
        if a['type'] in ['demand', 'standard']:
            d_nodes.append(n)
    except KeyError:
        pass

g_d = g.subgraph(d_nodes).copy()

In [14]:
# Find the supply sub-graph

s_nodes = []
for n, a in g.nodes(data=True):
    try:
        a['type']
        if a['type'] in ['supply', 'standard']:
            s_nodes.append(n)
    except KeyError:
        pass

g_supply = g.subgraph(s_nodes).copy()

In [15]:

def life_cycle_cost(data):
    """Note: operating cost is always empty"""
    if data["Capital cost"]["year_value"] is not None:
        capcost = data["Capital cost"]["year_value"]
    else:
        capcost = 0.0
    if data["Operating cost"]["year_value"] is not None:
        opcost = data["Operating cost"]["year_value"]
    else:
        opcost = 0.0
        
    lcc = capcost + opcost 
    return lcc
    
def market_share(alltechs, v=10):
    numtechs = len(alltechs)
    all_lccs  = {}
    marketshares = {}
    
    # alltechs is a dict of all (directly) competing tech subgraphs
    for tech, subgraph in alltechs.items():
        all_lccs[tech] = life_cycle_cost(subgraph)
    sum_v = sum(lcc**(-1.0*v) for lcc in all_lccs.values() if lcc > 0.0)
    
    for tech, val in all_lccs.items():
        if val > 0.0:
            marketshares[tech] = (val**(-1.0*v))/sum_v
        else:
            marketshares[tech] = 0.0
        
    return marketshares    

In [16]:
alberta = search_nodes( g_d, "Alberta")[0]
g_d.node[alberta]['2000'].keys()

dict_keys(['Service provided', 'Attribute', 'Price', 'Service requested'])

In [17]:
# get start values

population = g_d.node[alberta]['2000']["Attribute"]["Population"]["year_value"]
gdp = g_d.node[alberta]['2000']["Attribute"]["GDP"]["year_value"]
ngas_price = g_d.node[alberta]['2000']["Price"]["Natural Gas"]["year_value"]
electricity_price = g_d.node[alberta]['2000']["Price"]["Electricity"]["year_value"]

household_per_person = g_d.node[alberta]['2000']["Service requested"]["Residential"]["year_value"]
num_households = household_per_person * population

In [18]:
residential = search_nodes( g_d, "Residential")[0]
g_d.node[residential]['2000'].keys()

dict_keys(['Service provided', 'Price Multiplier', 'Price', 'Service requested', None])

In [19]:
ngas_multiplier = g_d.node[residential]['2000']["Price Multiplier"]["Natural Gas"]["year_value"]
electricity_multiplier = g_d.node[residential]['2000']["Price Multiplier"]["Electricity"]["year_value"]

electricity_price *= electricity_multiplier
ngas_price *= ngas_multiplier

In [20]:
# Start Buildings (parent of 1st tech compete)

buildings = search_nodes( g_d, "Buildings")[0]
g_d.node[buildings]['2000']['Market share'].keys()

dict_keys(['Single Family Detached', 'Single Family Attached', 'Apartment', 'Mobile'])

In [21]:
marketshare_famdet = g_d.node[buildings]['2000']['Market share']['Single Family Detached']["year_value"]
marketshare_famatt = g_d.node[buildings]['2000']['Market share']['Single Family Attached']["year_value"]
marketshare_apartment = g_d.node[buildings]['2000']['Market share']['Apartment']["year_value"]
marketshare_mobile = g_d.node[buildings]['2000']['Market share']['Mobile']["year_value"]

In [22]:
g_d.node[buildings]['2000']['Service requested'].keys()

dict_keys(['Single Family Detached', 'Single Family Attached', 'Apartment', 'Mobile'])

In [23]:
# PROBLEM: Dict gets rewritten everytime the key is the same, so we can't get any values before clothes washing (eg for shell)

# g_d.node[buildings]['2000']['Service requested']

In [24]:
# place values by hand due to problem above

space_per_famdet = 138.52 # g_d.node[buildings]['2000']['Service requested']['Single Family Detached']
space_per_famatt = 107.37
space_per_apartment = 84.76
space_per_mobile = 91.72

all_famdet = space_per_famdet*(marketshare_famdet * num_households)
all_famatt = space_per_famatt*(marketshare_famatt * num_households)
all_apartment = space_per_apartment*(marketshare_apartment * num_households)
all_mobile = space_per_mobile*(marketshare_mobile * num_households)

all_space = all_famdet + all_famatt + all_apartment + all_mobile

In [25]:
# shell

shell = search_nodes( g_d, "Shell")[0]
g_d.node[shell]['2000']['technologies'].keys()

dict_keys(['Pre-1960', 'Post-1960', 'Standard', 'R2000', 'LEED'])

In [26]:
pre1960_sg = g_d.node[shell]['2000']['technologies']['Pre-1960']
post1960_sg = g_d.node[shell]['2000']['technologies']['Post-1960']
standard_sg = g_d.node[shell]['2000']['technologies']['Standard']
R2000_sg = g_d.node[shell]['2000']['technologies']['R2000']
LEED_sg = g_d.node[shell]['2000']['technologies']['LEED']


# fetch v here
v_shell = g_d.node[shell]['2000']['Heterogeneity']['v']['year_value']

compete_shell = {"Pre-1960": pre1960_sg, 
                 "Post-1960": post1960_sg,
                 "Stadard": standard_sg,
                 "R2000": R2000_sg,
                 "LEED": LEED_sg}

In [27]:
marketshare_pre1960 = pre1960_sg["Market share"]["year_value"]
marketshare_post1960 = post1960_sg["Market share"]["year_value"]
marketshare_standard = standard_sg["Market share"]["year_value"]
marketshare_R2000 = R2000_sg["Market share"]["year_value"]
marketshare_LEED = LEED_sg["Market share"]["year_value"]


In [28]:
g_d.node[shell]['2000']['technologies'].keys()

dict_keys(['Pre-1960', 'Post-1960', 'Standard', 'R2000', 'LEED'])

In [29]:
# plug into 'output':is this correct

g_d.node[shell]['2000']['technologies']['Pre-1960']["Output"]["year_value"] = marketshare_pre1960*all_space
g_d.node[shell]['2000']['technologies']['Post-1960']["Output"]["year_value"] = marketshare_post1960*all_space
g_d.node[shell]['2000']['technologies']['Standard']["Output"]["year_value"] = marketshare_standard*all_space
g_d.node[shell]['2000']['technologies']['R2000']["Output"]["year_value"] = marketshare_R2000*all_space
g_d.node[shell]['2000']['technologies']['LEED']["Output"]["year_value"] = marketshare_LEED*all_space

# # here same as output: just multiplied by 1
pre1960 = g_d.node[shell]['2000']['technologies']['Pre-1960']['Service requested']["year_value"]*marketshare_pre1960*all_space
post1960 = g_d.node[shell]['2000']['technologies']['Post-1960']['Service requested']["year_value"]*marketshare_post1960*all_space
standard = g_d.node[shell]['2000']['technologies']['Standard']['Service requested']["year_value"]*marketshare_standard*all_space
R2000 = g_d.node[shell]['2000']['technologies']['R2000']['Service requested']["year_value"]*marketshare_R2000*all_space
LEED = g_d.node[shell]['2000']['technologies']['LEED']['Service requested']["year_value"]*marketshare_LEED*all_space

shellGJ = pre1960 + post1960 + standard + R2000 + LEED
# note: in 2000, only pre- and post-1960 are available

In [30]:
#Space Heating

space_heating = search_nodes(g_d, 'space heating')[0]
g_d.node[space_heating]['2000']['technologies'].keys()

dict_keys(['Electric baseboard', 'Furnace'])

In [31]:
electric_baseboard_sg = g_d.node[space_heating]['2000']['technologies']['Electric baseboard']
furnace_sg = g_d.node[space_heating]['2000']['technologies']['Furnace']
# fetch v here
v_space_heating = g_d.node[space_heating]['2000']['Heterogeneity']['v']['year_value']

compete_space_heating = {"Electric baseboard": electric_baseboard_sg, 
                         "Furnace": furnace_sg}

In [32]:
marketshare_electric_baseboard = electric_baseboard_sg["Market share"]["year_value"]
marketshare_furnace = furnace_sg["Market share"]["year_value"]

In [33]:
electric_baseboard_sg.keys()

dict_keys(['Available', 'Unavailable', 'Lifetime', 'Retirement', 'Discount rate_Financial', 'Discount rate_Retrofit', 'Discount rate_Revealed', 'Market share', 'Market share total_Max', 'Market share total_Min', 'Market share new_Max', 'Market share new_Min', 'Output', 'Capital cost', 'Declining capital cost_Limit', 'Declining capital cost_Existing stock', 'Declining capital cost_Progress Ratio', 'Declining capital cost_AEEI', 'Upfront fixed intangible cost', 'Upfront declining intangible cost_Initial', 'Upfront declining intangible cost_Rate', 'Upfront declining intangible cost_Shape', 'Annual fixed intangible cost', 'Annual declining intangible cost_Initial', 'Annual declining intangible cost_Rate', 'Annual declining intangible cost_Shape', 'Operating cost', 'Allocated cost', 'Service requested'])

In [34]:
g_d.node[space_heating]['2000']['technologies']['Electric baseboard']["Output"]["year_value"] = marketshare_electric_baseboard*shellGJ
g_d.node[space_heating]['2000']['technologies']['Furnace']["Output"]["year_value"] = marketshare_furnace*shellGJ

In [35]:
electric_baseboard_requested = g_d.node[space_heating]['2000']['technologies']['Electric baseboard']["Service requested"]["year_value"]*marketshare_electric_baseboard*shellGJ   
furnace_requested = g_d.node[space_heating]['2000']['technologies']['Furnace']["Service requested"]["year_value"]*marketshare_furnace*shellGJ   

In [36]:
# Furnace (sum with baseboard after)

furnace = search_nodes( g_d, "Furnace")[0]
g_d.node[furnace]['2000']['technologies'].keys()

dict_keys(['Natural Gas_existing', 'Heat pump_air source', 'Heat pump_ground source', 'Natural Gas_high efficiency'])

In [37]:
natural_gas_sg = g_d.node[furnace]['2000']['technologies']['Natural Gas_existing']
pump_air_sg = g_d.node[furnace]['2000']['technologies']['Heat pump_air source']
pump_ground_sg = g_d.node[furnace]['2000']['technologies']['Heat pump_ground source']
natural_gas_higheff_sg = g_d.node[furnace]['2000']['technologies']['Natural Gas_high efficiency']
# fetch v here
v_furnace = g_d.node[furnace]['2000']['Heterogeneity']['v']['year_value']

compete_furnace = {"Natual Gas": natural_gas_sg, 
                   "Pump Air": pump_air_sg,
                   "Pump Ground": pump_ground_sg,
                   "Natural Gas High Efficiency": natural_gas_higheff_sg}

In [38]:
marketshare_natural_gas = natural_gas_sg["Market share"]["year_value"]
marketshare_pump_air = pump_air_sg["Market share"]["year_value"]
marketshare_pump_ground = pump_ground_sg["Market share"]["year_value"]
marketshare_natural_gas_higheff = natural_gas_higheff_sg["Market share"]["year_value"]


In [39]:
g_d.node[furnace]['2000']['technologies']['Natural Gas_existing']["Output"]["year_value"] = marketshare_natural_gas*furnace_requested
g_d.node[furnace]['2000']['technologies']['Heat pump_air source']["Output"]["year_value"] = marketshare_pump_air*furnace_requested 
g_d.node[furnace]['2000']['technologies']['Heat pump_ground source']["Output"]["year_value"] = marketshare_pump_ground*furnace_requested 
g_d.node[furnace]['2000']['technologies']['Natural Gas_high efficiency']["Output"]["year_value"] = marketshare_natural_gas_higheff*furnace_requested 

In [40]:
natural_gas_requested = g_d.node[furnace]['2000']['technologies']['Natural Gas_existing']["Service requested"]["year_value"]*marketshare_natural_gas*furnace_requested   
natural_gas_requested += g_d.node[furnace]['2000']['technologies']['Natural Gas_high efficiency']["Service requested"]["year_value"]*marketshare_natural_gas_higheff*furnace_requested   

In [41]:
electricity_requested = g_d.node[furnace]['2000']['technologies']['Heat pump_air source']["Service requested"]["year_value"]*marketshare_pump_air*furnace_requested   
electricity_requested += g_d.node[furnace]['2000']['technologies']['Heat pump_ground source']["Service requested"]["year_value"]*marketshare_pump_ground*furnace_requested   

In [42]:
electricity_requested += electric_baseboard_requested 

In [61]:
#Supply: electricity

electricity = search_nodes( g, "Electricity")[0]
g.node[electricity]['2000'].keys()

dict_keys(['type', '2000', '2005', '2010', '2015', '2020', '2025', '2030', '2035', '2040', '2045', '2050'])

In [60]:
g[electricity]

AtlasView({'Canada.Alberta.Electricity.Generation': {'type': ['request_provide', 'structure']}, 'Canada.Alberta.Electricity.Transmission Losses': {'type': ['structure']}})

157772664.61883995

In [15]:
space_heating = search_nodes(g, 'space heating')[0]
furnace = search_nodes(g, 'Furnace')[0]

In [16]:
g.node[space_heating]['2000']['technologies'].keys()

dict_keys(['Electric baseboard', 'Furnace'])

In [17]:
electric_baseboard_sg = g.node[space_heating]['2000']['technologies']['Electric baseboard']
furnace_sg = g.node[space_heating]['2000']['technologies']['Furnace']
# fetch v here
v_space_heating = g.node[space_heating]['2000']['Heterogeneity']['v']['year_value']

compete_space_heating = {"Electric baseboard": electric_baseboard_sg, 
                         "Furnace": furnace_sg}

In [18]:
g.node[furnace]['2000']['technologies'].keys()

dict_keys(['Natural Gas_existing', 'Heat pump_air source', 'Heat pump_ground source', 'Natural Gas_high efficiency'])

In [19]:
# heat 

natural_gas_existing_sg = g.node[furnace]['2000']['technologies']['Natural Gas_existing']
heat_pump_air_source_sg = g.node[furnace]['2000']['technologies']['Heat pump_air source'] 
# heat_pump_ground_source_sg = g.node[furnace]['2000']['technologies']['Heat pump_ground source'] # unavail
natural_gas_high_efficiency_sg = g.node[furnace]['2000']['technologies']['Natural Gas_high efficiency']
v_furnace = g.node[furnace]['2000']['Heterogeneity']['v']['year_value']
compete_furnace = {"Natural Gas_existing": natural_gas_existing_sg,
                   "Heat pump_air source": heat_pump_air_source_sg,
                   "Natural Gas_high efficiency": natural_gas_high_efficiency_sg}

In [20]:
electricity = search_nodes(g_supply, 'electricity')[0]
natural_gas = search_nodes(g_supply, 'Natural Gas')[0]

In [21]:
electricity_sg = g_supply.node[electricity]['2000']
natural_gas_sg = g_supply.node[natural_gas]['2000']

In [73]:
# initialize service cost (2000 weighted LLC * MS)

In [24]:
# get initial values
natural_gas_existing_sg
# natural_gas_high_efficiency_sg




{'Available': {'source': None,
  'branch': None,
  'unit': 'Date',
  'year_value': 2000.0},
 'Unavailable': {'source': None,
  'branch': None,
  'unit': 'Date',
  'year_value': 2000.0},
 'Lifetime': {'source': None,
  'branch': None,
  'unit': 'Years',
  'year_value': 18.0},
 'Retirement': {'source': None,
  'branch': None,
  'unit': 'Years',
  'year_value': None},
 'Discount rate_Financial': {'source': None,
  'branch': None,
  'unit': '%',
  'year_value': 0.25},
 'Discount rate_Retrofit': {'source': None,
  'branch': None,
  'unit': '%',
  'year_value': None},
 'Discount rate_Revealed': {'source': None,
  'branch': None,
  'unit': '%',
  'year_value': None},
 'Market share': {'source': None,
  'branch': None,
  'unit': '%',
  'year_value': 0.94},
 'Market share total_Max': {'source': None,
  'branch': None,
  'unit': '%',
  'year_value': None},
 'Market share total_Min': {'source': None,
  'branch': None,
  'unit': '%',
  'year_value': None},
 'Market share new_Max': {'source': None,

In [95]:
a = market_share(compete_furnace)

In [96]:
a

{'Natural Gas_existing': 0.9990243902439024,
 'Natural Gas_high efficiency': 0.000975609756097561}

In [ ]:
class Demand:
    '''
    
    - **kwargs: dict of named arguments in "argname": value pairs to specify used economic indicators (GDP, population, etc)
    '''

    def __init__(self, data, v=10, keyfuel="kind", return_all_info=False, **kwargs):
        self.keyfuel = keyfuel
        
        self.dtechs = data["demand_step"]
        self.dfuels = data["supply_step"]
        
        lccs = self.lcc_over_tech()
        
        if return_all_info:
            tech_demand, tech_mshares = self.quantity_demanded(lccs, v=v, return_all_info=True)
            fuel_demand = self.demand_by_fuel(tech_demand)
            self.fuel_demand, self.tech_demand, self.tech_mshares = fuel_demand, tech_demand, tech_mshares
        else:
            tech_demand = self.quantity_demanded(lccs, v=v, return_all_info=False)
            fuel_demand = self.demand_by_fuel(tech_demand)
            self.fuel_demand = fuel_demand
        
        

        
    def get_quantity(self, ms, fuel_quantity):
        demand = ms * fuel_quantity
        return demand
    
    def lcc_over_tech(self, keytech = "kind", keysubtech = "subtypes",
                            keyprice = "price", keycapcost = "cap_cost"):
        # TODO: use some kwargs in lcc method in case there are arbitrary numbers of params
        lccs = {}
        # techdict is the entire subtree for the kth competing tech (demand side)
        for techdict in self.dtechs:
            # get the tech name (key) to use as a label for lcc
            tech_k = techdict[keytech]
            lccs[tech_k] = {}
            # fetch all attributes/values for the (sub)tech that demands a fuel
            # by `subtech` I mean e.g.: tech: furnace, subtech: type of furnace
            subtechs = techdict[keysubtech]            
            # loop over `subtechs`, get subtech name (key) and vals (all attributes associated w/ subtech)
            for subtech, vals in subtechs.items():
                # unpack components needed for lcc
                price = vals[keyprice]
                cc = vals[keycapcost]
                lccs[tech_k][subtech] = self.life_cycle_cost(last_price=price, cap_cost=cc) # kwargs here
        return lccs        
                
                
    def quantity_demanded(self, lccs, v=10, return_all_info=True, keytech="kind", 
                                keysubtech="subtypes", keyquant="total_demand"):
        mshares, quants = {}, {}

        # techdict is the entire subtree for the kth competing tech (demand side)            
        for techdict in self.dtechs:
            # fetch all attributes/values for the (sub)tech that demands a fuel
            # by `subtech` I mean e.g.: tech: furnace, subtech: type of furnace
            subtechs = techdict[keysubtech]
            # loop over `subtechs`, get subtech name (key) and vals (all attributes associated w/ subtech)                
                
            for tech, vals in subtechs.items():
                mshares[tech] = self.market_share(lccs[techdict[keytech]], tech, v=v)
                quants[tech] = self.get_quantity(mshares[tech], techdict[keyquant])     
                    
        if return_all_info:
            return quants, mshares
        else:
            return quants
        
    def demand_by_fuel(self, quants, keytechfuel="fuel", keysubtech="subtypes"):
        by_fuel, sum_quants = {}, {}

        # get only numbers for a given fuel, in both techs
        for fkind in self.dfuels:
            cur_fuel = fkind[self.keyfuel]
            by_fuel[cur_fuel] = filter_obs(data, fuel_type=cur_fuel, val="fuel", verbose=False)
            sum_quants[cur_fuel] = sum(quants[subtype] for subtype in by_fuel[cur_fuel].keys())
        return sum_quants    
                    



#### Define classes for Supply and Demand:

In [6]:
class Demand:
    '''
    
    - **kwargs: dict of named arguments in "argname": value pairs to specify used economic indicators (GDP, population, etc)
    '''

    def __init__(self, data, v=10, keyfuel="kind", return_all_info=False, **kwargs):
        self.keyfuel = keyfuel
        
        self.dtechs = data["demand_step"]
        self.dfuels = data["supply_step"]
        
        lccs = self.lcc_over_tech()
        
        if return_all_info:
            tech_demand, tech_mshares = self.quantity_demanded(lccs, v=v, return_all_info=True)
            fuel_demand = self.demand_by_fuel(tech_demand)
            self.fuel_demand, self.tech_demand, self.tech_mshares = fuel_demand, tech_demand, tech_mshares
        else:
            tech_demand = self.quantity_demanded(lccs, v=v, return_all_info=False)
            fuel_demand = self.demand_by_fuel(tech_demand)
            self.fuel_demand = fuel_demand
        
        
    ## Methods for demand side calculations 
    def life_cycle_cost(self, last_price, cap_cost=0):
        lcc = (last_price + cap_cost)
        return lcc
    
    def market_share(self, lcc, techname, v=10):
        # lcc is a dict
        sum_v = sum(lcc[item]**(-1.0*v) for item in lcc)
        ms = (lcc[techname]**(-1.0*v))/sum_v
        return ms
        
    def get_quantity(self, ms, fuel_quantity):
        demand = ms * fuel_quantity
        return demand
    
    def lcc_over_tech(self, keytech = "kind", keysubtech = "subtypes",
                            keyprice = "price", keycapcost = "cap_cost"):
        # TODO: use some kwargs in lcc method in case there are arbitrary numbers of params
        lccs = {}
        # techdict is the entire subtree for the kth competing tech (demand side)
        for techdict in self.dtechs:
            # get the tech name (key) to use as a label for lcc
            tech_k = techdict[keytech]
            lccs[tech_k] = {}
            # fetch all attributes/values for the (sub)tech that demands a fuel
            # by `subtech` I mean e.g.: tech: furnace, subtech: type of furnace
            subtechs = techdict[keysubtech]            
            # loop over `subtechs`, get subtech name (key) and vals (all attributes associated w/ subtech)
            for subtech, vals in subtechs.items():
                # unpack components needed for lcc
                price = vals[keyprice]
                cc = vals[keycapcost]
                lccs[tech_k][subtech] = self.life_cycle_cost(last_price=price, cap_cost=cc) # kwargs here
        return lccs        
                
                
    def quantity_demanded(self, lccs, v=10, return_all_info=True, keytech="kind", 
                                keysubtech="subtypes", keyquant="total_demand"):
        mshares, quants = {}, {}

        # techdict is the entire subtree for the kth competing tech (demand side)            
        for techdict in self.dtechs:
            # fetch all attributes/values for the (sub)tech that demands a fuel
            # by `subtech` I mean e.g.: tech: furnace, subtech: type of furnace
            subtechs = techdict[keysubtech]
            # loop over `subtechs`, get subtech name (key) and vals (all attributes associated w/ subtech)                
                
            for tech, vals in subtechs.items():
                mshares[tech] = self.market_share(lccs[techdict[keytech]], tech, v=v)
                quants[tech] = self.get_quantity(mshares[tech], techdict[keyquant])     
                    
        if return_all_info:
            return quants, mshares
        else:
            return quants
        
    def demand_by_fuel(self, quants, keytechfuel="fuel", keysubtech="subtypes"):
        by_fuel, sum_quants = {}, {}

        # get only numbers for a given fuel, in both techs
        for fkind in self.dfuels:
            cur_fuel = fkind[self.keyfuel]
            by_fuel[cur_fuel] = filter_obs(data, fuel_type=cur_fuel, val="fuel", verbose=False)
            sum_quants[cur_fuel] = sum(quants[subtype] for subtype in by_fuel[cur_fuel].keys())
        return sum_quants    
                    



In [7]:
class Supply:
    def __init__(self, data, v=10, keyfuel="kind", return_all_info=True, **kwargs):
        self.keyfuel = keyfuel
        
        self.sfuels = data["supply_step"]
        lccs = self.lcc_over_production()
        
        if return_all_info:
            self.tot_price, self.price, self.sup_mshares = self.get_fuelprice(lccs, v)
        else:
            self.tot_price = self.get_fuelprice(lccs, v)            
            
            
    ## Methods for calculating demand side 
    def life_cycle_cost(self, last_price, cap_cost=0):
        lcc = (last_price + cap_cost)
        return lcc
        
    def market_share(self, lcc, prodname, v=10):
        # lcc is a dict
        sum_v = sum(lcc[item]**(-1.0*v) for item in lcc)
        ms = (lcc[prodname]**(-1.0*v))/sum_v
        return ms
        
    def get_price(self, ms, lcc):
        price = ms*lcc       
        return price
                
        
        
    def lcc_over_production(self, keyprod = "subtypes", keyprice = "price", keycapcost = "cap_cost"):  
        lccs = {}        
        for fueldict in self.sfuels:
            # loop through each fuel, fueldict is entire subtree below each type of fuel
            fueltype = fueldict[self.keyfuel]
            lccs[fueltype] = {}
            
            for prod, vals in fueldict[keyprod].items():
                # loop through each production type (say 'hydro' if fuel is 'electricity')
                lccs[fueltype][prod] = self.life_cycle_cost(last_price=vals[keyprice], cap_cost=vals[keycapcost])
        return lccs    
    
    def get_fuelprice(self, lccs, v = 10, keyprod = "subtypes", keyprice = "price", 
                            keycapcost = "cap_cost", return_all_info = True): 
        # distinguish the 2 types of lccs (one for added, one for price)
        lcc_model, mshares, price, tot_price = {}, {}, {}, {}
        
        for fueldict in self.sfuels:
            # loop through each fuel, fueldict is entire subtree below each type of fuel
            fueltype = fueldict[self.keyfuel]                          
            lcc_model[fueltype], mshares[fueltype], price[fueltype] = {}, {}, {}
            for prod, vals in fueldict[keyprod].items():               
                lcc_model[fueltype][prod] = self.life_cycle_cost(last_price=vals[keyprice], cap_cost=vals[keycapcost])
                mshares[fueltype][prod] = self.market_share(lccs[fueltype], prod, v=v)                
                price[fueltype][prod] = self.get_price(mshares[fueltype][prod], lcc_model[fueltype][prod])    # (total_quants["electricity"])   
            tot_price[fueltype] = sum(price[fueltype][p]/len(fueldict[keyprod].keys()) for p in fueldict[keyprod].keys())
            # TODO!! check averaging method
            
        if return_all_info:
            return tot_price, price, mshares 
        else:
            return tot_price
            
            

In [8]:
class SupplyDemand:
    '''
    SUPPLY-DEMAND MODEL:
        - Demand: estimate Quantity
        - Supply: estimate Price
        - Repeat using updated prices    
    '''    
    def __init__(self, data, verbose=True,
                             demandkey="demand_step",
                             supplykey="supply_step",
                             reskeys=("dem_price_by_fuel", "quant_by_fuel", "sup_price_by_fuel"),
                             infokeys=("fuels", "price_diff"),
                             keyfuel="kind"):
        self.data = data   
        self.techs = data[demandkey]
        self.fuels = data[supplykey]
        self.tempdata = deepcopy(data) # check in case data is unsafe to change
        self.keyfuel = keyfuel
        self.demandkey = demandkey
        self.supplykey = supplykey
        
        # may want to make keys editable from args instead
        self.reskeys = reskeys
        self.results = DictAppend()
        self.infokeys = infokeys
        self.info = DictAppend()
        
        self.setup()
        
    def setup(self):
        
        self.demand = Demand(self.data)
        quant_hat = self.demand.fuel_demand       

        init_price = {ft[self.keyfuel]: find_price_for_fuel( fuel_type = ft[self.keyfuel], data = self.techs ) for ft in self.fuels}
        self.supply = Supply(self.data)
        price_hat = self.supply.tot_price
        new_res = [init_price, quant_hat, price_hat]

        for num, res in enumerate(new_res):
            for resname, resval in res.items():
                self.results[self.reskeys[num]][resname].append(resval)
                
                
                
    def iterate(self, relative_tol=0.05, max_iter=10, n_iter=0):
        price_diff, price_thres, ge_thres = {}, {}, {}
        
        print(self.fuels)
        
        for ft in self.fuels: 
            # storing fuels used in current competition cluster
            f = ft[self.keyfuel]
            self.info[self.infokeys[0]].append(ft[self.keyfuel]) 

            # last demand price recoreded for fuel f (here just initial demand price)
            last_price = self.results[self.reskeys[0]][f][-1] 
            # last supply price recoreded for fuel f
            cur_price = self.results[self.reskeys[2]][f][-1]
            price_diff[f] = np.abs(last_price - cur_price)
            price_thres[f] = cur_price * relative_tol
            ge_thres[f] = price_diff[f] > price_thres[f]
            # store price difference
            self.info[self.infokeys[1]].append( price_diff )
        
        while any(ge_thres) and n_iter <= max_iter:
            # iterate until price diff >= relative_thres and until max number of iters reached
            for fuel, is_ge in ge_thres.items():
                new_price = self.results[self.reskeys[2]][fuel][-1]
                # change price for fuel belonging to all techs in initial `dataset`
                change_price_for_fuel( self.data[self.demandkey], fuel, new_price )

            
            self.demand = Demand(self.data)
            quant_hat = self.demand.fuel_demand
            new_price = {ft[self.keyfuel]: find_price_for_fuel( fuel_type = ft[self.keyfuel], data = self.techs ) for ft in self.fuels}
            self.supply = Supply(self.data)
            
            price_hat = self.supply.tot_price
            
            for ft, val in new_price.items():
                self.results[self.reskeys[0]][ft].append(val)
            for ft, val in quant_hat.items():    
                self.results[self.reskeys[1]][ft].append(val)
            for ft, val in price_hat.items():
                self.results[self.reskeys[2]][ft].append(val)          
                
            #TODO (done here oct 13 pm, check price diff vals, check label fuck up)
            for f in self.info["fuels"]:
                last_price = self.results["dem_price_by_fuel"][f][-1]    
                cur_price = self.results["sup_price_by_fuel"][f][-1]
                price_diff[f] = np.abs(last_price - cur_price)
                price_thres[f] = cur_price * relative_tol
                ge_thres[f] = price_diff[f] > price_thres[f]
                
            self.info["price_diff"].append( price_diff )
            n_iter += 1

                


In [9]:
sd = SupplyDemand(data)
sd.iterate()

[{'kind': 'electricity', 'leaf': True, 'supply': 5000, 'subtypes': {'wind': {'price': 110, 'cap_cost': 2}, 'hydro': {'price': 100, 'cap_cost': 5}, 'rocks': {'price': 80, 'cap_cost': 10}}}, {'kind': 'gas', 'leaf': True, 'supply': 3000, 'subtypes': {'gas1': {'price': 150, 'cap_cost': 10}, 'gas2': {'price': 120, 'cap_cost': 0}}}]


In [10]:
sd.results

{'dem_price_by_fuel': {'electricity': [200,
   31.42768280241502,
   31.42768280241502,
   31.42768280241502,
   31.42768280241502,
   31.42768280241502,
   31.42768280241502,
   31.42768280241502,
   31.42768280241502,
   31.42768280241502,
   31.42768280241502,
   31.42768280241502],
  'gas': [150,
   61.06622728811647,
   61.06622728811647,
   61.06622728811647,
   61.06622728811647,
   61.06622728811647,
   61.06622728811647,
   61.06622728811647,
   61.06622728811647,
   61.06622728811647,
   61.06622728811647,
   61.06622728811647]},
 'quant_by_fuel': {'electricity': [11212.77741142919,
   18995.03215918419,
   18995.03215918419,
   18995.03215918419,
   18995.03215918419,
   18995.03215918419,
   18995.03215918419,
   18995.03215918419,
   18995.03215918419,
   18995.03215918419,
   18995.03215918419,
   18995.03215918419],
  'gas': [7787.222588570809,
   4.967840815811117,
   4.967840815811117,
   4.967840815811117,
   4.967840815811117,
   4.967840815811117,
   4.9678408158111

In [11]:
demPrice = sd.results["dem_price_by_fuel"]["electricity"]
supPrice = sd.results['sup_price_by_fuel']["electricity"]
quants = sd.results["quant_by_fuel"]["electricity"]

In [12]:
sd.info

{'fuels': ['electricity', 'gas'],
 'price_diff': [{'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0}]}